In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
import time
import sys

#### Scaled Dot-Product Attention

When we take a dot product of query and key values, the variance of the result of the dot product may be scaled by $d$, which is the dimension of the key/query vectors. To ensure that the variance of the dot product still remains the same regardless of vector length, the scaled dot-product attention scoring function is used.

Assuming $\mathbf{Q} \in \mathbb{R}^{n \times d}$, keys $\mathbf{K}\in \mathbb{R}^{m \times d}$, values $\mathbf{V}\in \mathbb{R}^{m \times v}$ we do

$$\text{softmax} \left(\frac{\mathbf{Q}\mathbf{K}^{T}}{\sqrt{d}}\right)\mathbf{V} \quad \in \mathbb{R}^{n\times v}$$

In [2]:
class ScaledDotProductAttention(nn.Module):
    def __init__(self, d_k):
        super(ScaledDotProductAttention, self).__init__()
        self.scale_factor = np.sqrt(d_k)
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, q, k, v, attn_mask=None):
        # q: [b_size x len_q x d_k]
        # k: [b_size x len_k x d_k]
        # v: [b_size x len_v x d_v] note: (len_k == len_v)
        # Batch-Matrix Multiplication
        attn = torch.bmm(q, k.transpose(1, 2)) / self.scale_factor  # attn: [b_size x len_q x len_k]
        if attn_mask is not None:
        #    assert attn_mask.size() == attn.size()
            attn.data.masked_fill_(attn_mask==0, -1e32)

        attn = self.softmax(attn )
        outputs = torch.bmm(attn, v) # outputs: [b_size x len_q x d_v]
        return outputs, attn

From : http://nlp.seas.harvard.edu/2018/04/01/attention.html#position-wise-feed-forward-networks

In addition to attention sub-layers, each of the layers in our encoder and decoder contains a fully connected feed-forward network, which is applied to each position separately and identically. This consists of two linear transformations with a ReLU activation in between.

$$FFN(x)=max(0,xW_1+b_1)W_2+b_2$$

While the linear transformations are the same across different positions, they use different parameters from layer to layer. Another way of describing this is as two convolutions with kernel size 1. The dimensionality of input and output is $d_{model}$, and the inner-layer has dimensionality $d_{ff}$.

Layer normalization (LayerNorm) is a technique to normalize the distributions of intermediate layers. It enables smoother gradients, faster training, and better generalization accuracy

In [3]:
class PositionwiseFeedForward(nn.Module):
    "Implements FFN equation."
    def __init__(self, d_model, d_ff):
        super(PositionwiseFeedForward, self).__init__()
        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.layer_norm = nn.LayerNorm(d_model)

    def forward(self, x):
        residual = x # inputs: [b_size x len_q x d_model]
        outputs = self.w_2(F.relu(self.w_1(x)))
        return self.layer_norm(residual + outputs)

In [4]:
class _MultiHeadAttention(nn.Module):
    def __init__(self, d_model, n_heads):
        super(_MultiHeadAttention, self).__init__()

        self.d_k = d_model // n_heads
        self.d_v = d_model // n_heads
        self.d_model = d_model
        self.n_heads = n_heads

        self.w_q = nn.Parameter(torch.FloatTensor(n_heads, d_model, self.d_k))
        self.w_k = nn.Parameter(torch.FloatTensor(n_heads, d_model, self.d_k))
        self.w_v = nn.Parameter(torch.FloatTensor(n_heads, d_model, self.d_v))

        self.attention = ScaledDotProductAttention(self.d_k)

    def forward(self, q, k, v, attn_mask=None, is_adj=True):
        (d_k, d_v, d_model, n_heads) = (self.d_k, self.d_v, self.d_model, self.n_heads)
        b_size = k.size(0)

        q_s = q.repeat(n_heads, 1, 1).view(n_heads, -1, d_model)  # [n_heads x b_size * len_q x d_model]
        k_s = k.repeat(n_heads, 1, 1).view(n_heads, -1, d_model)  # [n_heads x b_size * len_k x d_model]
        v_s = v.repeat(n_heads, 1, 1).view(n_heads, -1, d_model)  # [n_heads x b_size * len_v x d_model]

        q_s = torch.bmm(q_s, self.w_q).view(b_size * n_heads, -1, d_k)  # [b_size * n_heads x len_q x d_k]
        k_s = torch.bmm(k_s, self.w_k).view(b_size * n_heads, -1, d_k)  # [b_size * n_heads x len_k x d_k]
        v_s = torch.bmm(v_s, self.w_v).view(b_size * n_heads, -1, d_v)  # [b_size * n_heads x len_v x d_v]

        if attn_mask is not None:
            if is_adj:
                outputs, attn = self.attention(q_s, k_s, v_s, attn_mask=attn_mask.repeat(n_heads, 1, 1))
            else:
                outputs, attn = self.attention(q_s, k_s, v_s, attn_mask=attn_mask.unsqueeze(1).repeat(n_heads, 1, 1))
        else:
            outputs, attn = self.attention(q_s, k_s, v_s, attn_mask=None)

        return torch.split(outputs, b_size, dim=0), attn

In [5]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, n_heads):
        super(MultiHeadAttention, self).__init__()

        self.d_k = d_model // n_heads
        self.attention = _MultiHeadAttention(d_model, n_heads)
        self.proj = nn.Linear(n_heads * self.d_k, d_model)
        self.layer_norm = nn.LayerNorm(d_model)

    def forward(self, q, k, v, attn_mask = None, is_adj = True):
        # q: [b_size x len_q x d_model]
        # k: [b_size x len_k x d_model]
        # v: [b_size x len_v x d_model] note (len_k == len_v)
        residual = q
        # outputs: a list of tensors of shape [b_size x len_q x d_v] (length: n_heads)
        outputs, attn = self.attention(q, k, v, attn_mask=attn_mask, is_adj=is_adj)
        # concatenate 'n_heads' multi-head attentions
        outputs = torch.cat(outputs, dim=-1)
        # project back to residual size, result_size = [b_size x len_q x d_model]
        outputs = self.proj(outputs)

        return self.layer_norm(residual + outputs), attn

A single Encoder Layer consists of One MultiHead Attention Layer and one PositionWiseFeedForward layer. Its forward pass consists of Encoder input, Recursive Encoder input and self attention mask

In [7]:
class EncoderLayer(nn.Module):
    def __init__(self, d_model, d_ff, n_heads):
        super(EncoderLayer, self).__init__()

        self.enc_self_attn = MultiHeadAttention(d_model, n_heads)
        self.pos_ffn = PositionwiseFeedForward(d_model, d_ff)

    def forward(self, enc_inp, rec_enc_inp, self_attn_mask):
        enc_outputs, attn = self.enc_self_attn(enc_inp, rec_enc_inp, enc_inp, attn_mask=self_attn_mask)
        enc_outputs = self.pos_ffn(enc_outputs)

        return enc_outputs, attn

So the encoder here consists of:
1. A layer for static features to number of hidden layers / 2
2. A layer for dynamic features to number of hidden layers / 2.

These layers constitute the input layers. Here's where we give input to the network.

Then, we have the hidden layers. (May be Embeddings? I don't know.). But they are a Module List of Layers each of type EncoderLayer, having as input the hidden_size and output as $d_{ff}$, which are the dimensions of the feed forward layers. 

In [6]:
class Encoder(nn.Module):
    def __init__(self, features_dim, dfeatures_dim, hidden_size, args):
        super(Encoder, self).__init__()

        n_heads = args.n_heads # number of heads
        d_ff = args.ff_dim # feed_forward_hidden
        n_layers = args.n_layers # number of Layers

        self.L1 = nn.Linear(features_dim, hidden_size//2) # for static features
        self.L2 = nn.Linear(dfeatures_dim, hidden_size//2) # for dynamic features

        self.layers = nn.ModuleList([EncoderLayer(hidden_size, d_ff, n_heads) for _ in range(n_layers)])

    def forward(self, emb_inp, rec_inp, mask, dummy_arg):
        for layer in self.layers:
            emb_inp, _ = layer(emb_inp, rec_inp, mask)

        return emb_inp


In [8]:
class Decoder(nn.Module):
    def __init__(self, hidden_size):
        super(Decoder, self).__init__()

        self.W1 = nn.Linear(hidden_size, hidden_size, bias=False)
        self.W2 = nn.Linear(hidden_size, hidden_size)
        self.V = nn.Parameter(torch.zeros((hidden_size, 1), requires_grad=True))

        self.first_h_0 = nn.Parameter(torch.FloatTensor(1, hidden_size), requires_grad=True)
        self.first_h_0.data.uniform_(-(1. / math.sqrt(hidden_size)), 1. / math.sqrt(hidden_size))

        self.c0 = nn.Parameter(torch.FloatTensor( 1, hidden_size),requires_grad=True)
        self.c0.data.uniform_(-(1. / math.sqrt(hidden_size)), 1. / math.sqrt(hidden_size))

        self.hidden_0 = (self.first_h_0, self.c0)

        self.lstm = nn.LSTMCell(hidden_size, hidden_size)


    def forward(self, input, hidden, enc_outputs, mask):
        hidden = self.lstm(input, hidden)
        w1e = self.W1(enc_outputs)
        w2h = self.W2(hidden[0]).unsqueeze(1)
        u = torch.tanh(w1e + w2h)
        a = u.matmul(self.V)
        a = 10*torch.tanh(a).squeeze(2)

        policy = F.softmax(a + mask.float().log(), dim=1)

        return policy, hidden

So the Pointer Network here takes as input 1) static features, 2) Dynamic Features 3) Number of dimensions of the hidden layers. It contains a decoder block and an encoder block.

The parameters are initialized with Xavier initialization.

The goal of Xavier Initialization is to initialize the weights such that the variance of the activations are the same across every layer. This constant variance helps prevent the gradient from exploding or vanishing. (Taken from https://cs230.stanford.edu/section/4/)

(Xavier initialization is designed to work well with tanh or sigmoid activation functions. For ReLU activations, look into He initialization, which follows a very similar derivation. https://cs230.stanford.edu/section/4/ My note: Need to check this further)



In [9]:
class RecPointerNetwork(nn.Module):

    def __init__(self, features_dim, dfeatures_dim, hidden_dim, args):
        super(RecPointerNetwork, self).__init__()

        self.features_dim = features_dim
        self.dfeatures_dim = dfeatures_dim
        self.use_checkpoint = args.use_checkpoint
        self.hidden_dim = hidden_dim
        self.decoder = Decoder(hidden_dim)
        self.encoder = Encoder(features_dim, dfeatures_dim, hidden_dim, args)
        # see https://discuss.pytorch.org/t/checkpoint-with-no-grad-requiring-inputs-problem/19117/11
        self.dummy_tensor = torch.ones(1, dtype=torch.float32, requires_grad=True)

        self._initialize_parameters()

    def _initialize_parameters(self):
        for name, param in self.named_parameters():
            if len(param.shape) > 1:
                nn.init.xavier_uniform_(param)

    def _load_model_weights(self, path_string, device):
        self.load_state_dict(torch.load(path_string, map_location=device))


    def forward(self, enc_inputs, enc_hidden, adj_mask, dec_input, dec_hidden, mask, first_step=False):
        policy, dec_hidden, enc_outputs = self._one_step(enc_inputs, enc_hidden, adj_mask, dec_input, dec_hidden, mask, first_step)
        return policy, dec_hidden, enc_outputs

    def _one_step(self, enc_inputs, enc_hidden, adj_mask, dec_input, dec_hidden, mask, first_step):
        if self.use_checkpoint:
            enc_outputs = checkpoint(self.encoder, enc_inputs, enc_hidden, adj_mask, self.dummy_tensor)
        else:
            enc_outputs = self.encoder(enc_inputs, enc_hidden, adj_mask, self.dummy_tensor)

        if first_step:
            return  None, None, enc_outputs
        else:
            policy, dec_hidden = self.decoder(dec_input, dec_hidden, enc_outputs, mask)
            return policy, dec_hidden, enc_outputs

    def sta_emb(self, sta_inp):
        return torch.tanh(self.encoder.L1(sta_inp))

    def dyn_emb(self, dyn_inp):
        return torch.tanh(self.encoder.L2(dyn_inp))


In [10]:
class BeamSearch(nn.Module):
    def __init__(self, neuralnet, args):
        super(BeamSearch, self).__init__()

        self.device = args.device
        self.neuralnet = neuralnet
        self.dyn_feat = DynamicFeatures(args)
        self.lookahead = Lookahead(args)
        self.mu = ModelUtils(args)

    def forward(self, inputs, data_scaled, start_time, dist_mat, infer_type, beam_size):
        self.beam_size = beam_size
        _, sequence_size, input_size = inputs.size()


        # first step  - node 0
        bpresent_time = start_time*torch.ones(1, 1, device=self.device)

        mask = torch.ones(1, sequence_size, device=self.device, requires_grad=False, dtype= torch.uint8)
        bpres_actions = torch.zeros(1, dtype=torch.int64,device=self.device)
        beam_idx = torch.arange(0, 1, device=self.device)

        done, mask = self.mu.feasibility_control(inputs.expand(beam_idx.shape[0], -1, -1),
                                                 mask, dist_mat, bpres_actions, bpresent_time,
                                                 torch.arange(0, mask.shape[0], device=self.device),
                                                 first_step=True)
        adj_mask = self.lookahead.adjacency_matrix(inputs.expand(beam_idx.shape[0], -1, -1),
                                                   mask, dist_mat, bpres_actions, bpresent_time)

        h_0, c_0 = self.neuralnet.decoder.hidden_0
        dec_hidden = (h_0.expand(1, -1), c_0.expand(1, -1))

        step = 0

        # encoder first forward pass
        bdata_scaled = data_scaled.expand(1,-1,-1)
        sum_log_probs = torch.zeros(1, device=self.device).float()

        bdyn_inputs = self.dyn_feat.make_dynamic_feat(inputs.expand(1,-1,-1), bpresent_time, bpres_actions, dist_mat, beam_idx)
        emb1 = self.neuralnet.sta_emb(bdata_scaled)
        emb2 = self.neuralnet.dyn_emb(bdyn_inputs)
        enc_inputs = torch.cat((emb1, emb2), dim=2)

        _, _, enc_outputs = self.neuralnet(enc_inputs, enc_inputs, adj_mask, enc_inputs, dec_hidden, mask, first_step=True)

        decoder_input = enc_outputs[beam_idx, bpres_actions]

        done, mask = self.mu.feasibility_control(inputs.expand(beam_idx.shape[0], -1, -1),
                                                 mask, dist_mat, bpres_actions, bpresent_time,
                                                 torch.arange(0, mask.shape[0], device=self.device))
        adj_mask = self.lookahead.adjacency_matrix(inputs.expand(beam_idx.shape[0], -1, -1),
                                                   mask, dist_mat, bpres_actions, bpresent_time)

        # encoder/decoder forward pass
        bdyn_inputs = self.dyn_feat.make_dynamic_feat(inputs.expand(beam_idx.shape[0], -1, -1), bpresent_time, bpres_actions, dist_mat, beam_idx)
        emb2 = self.neuralnet.dyn_emb(bdyn_inputs)
        enc_inputs = torch.cat((emb1,emb2), dim=2)

        policy, dec_hidden, enc_outputs = self.neuralnet(enc_inputs, enc_outputs, adj_mask, decoder_input, dec_hidden, mask)

        future_actions, log_probs, beam_idx = self.select_actions(policy, sum_log_probs, mask, infer_type)
        # info update
        h_step = torch.index_select(dec_hidden[0], dim=0, index = beam_idx)
        c_step = torch.index_select(dec_hidden[1], dim=0, index = beam_idx)
        dec_hidden = (h_step,c_step)

        mask = torch.index_select(mask, dim=0, index=beam_idx)
        bpresent_time = torch.index_select(bpresent_time, dim=0, index=beam_idx)
        bpres_actions = torch.index_select(bpres_actions, dim=0, index=beam_idx)
        enc_outputs  = torch.index_select(enc_outputs, dim=0, index=beam_idx)
        sum_log_probs = torch.index_select(sum_log_probs, dim=0, index=beam_idx)

        emb1 = torch.index_select(emb1, dim=0, index=beam_idx)

        # initialize buffers
        bllog_probs = torch.zeros(bpres_actions.shape[0], sequence_size, device=self.device).float()
        blactions = torch.zeros(bpres_actions.shape[0], sequence_size, device=self.device).long()

        sum_log_probs += log_probs.squeeze(0).detach()

        blactions[:, step] = bpres_actions

        final_log_probs, final_actions, lstep_mask = [], [], []

        # Starting the trip
        while not done:

            future_actions = future_actions.squeeze(0)

            beam_size = bpres_actions.shape[0]
            bpres_actions, bpresent_time, bstep_mask = \
                self.mu.one_step_update(inputs.expand(beam_size, -1, -1), dist_mat,
                                        bpres_actions, future_actions, bpresent_time,
                                        torch.arange(0,beam_size,device=self.device),
                                        beam_size)

            bllog_probs[:, step] = log_probs
            blactions[:, step+1] = bpres_actions
            step+=1

            done, mask = self.mu.feasibility_control(inputs.expand(beam_idx.shape[0], -1, -1),
                                                     mask, dist_mat, bpres_actions, bpresent_time,
                                                     torch.arange(0, mask.shape[0], device=self.device))
            adj_mask = self.lookahead.adjacency_matrix(inputs.expand(beam_idx.shape[0], -1, -1),
                                                       mask, dist_mat, bpres_actions, bpresent_time)

            active_beam_idx = torch.nonzero(mask[:, -1], as_tuple=False).squeeze(1)
            end_beam_idx = torch.nonzero((mask[:, -1]==0), as_tuple=False).squeeze(1)

            if end_beam_idx.shape[0]>0:

                final_log_probs.append(torch.index_select(bllog_probs, dim=0, index=end_beam_idx))
                final_actions.append(torch.index_select(blactions, dim=0, index=end_beam_idx))

                # ending seq info update
                h_step = torch.index_select(dec_hidden[0], dim=0, index = active_beam_idx)
                c_step = torch.index_select(dec_hidden[1], dim=0, index = active_beam_idx)
                dec_hidden = (h_step,c_step)

                mask = torch.index_select(mask, dim=0, index=active_beam_idx)
                adj_mask = torch.index_select(adj_mask, dim=0, index=active_beam_idx)

                bpresent_time = torch.index_select(bpresent_time, dim=0, index=active_beam_idx)
                bpres_actions = torch.index_select(bpres_actions, dim=0, index=active_beam_idx)
                enc_outputs  = torch.index_select(enc_outputs, dim=0, index=active_beam_idx)

                emb1 = torch.index_select(emb1, dim=0, index=active_beam_idx)

                blactions = torch.index_select(blactions, dim=0, index=active_beam_idx)
                bllog_probs = torch.index_select(bllog_probs, dim=0, index=active_beam_idx)
                sum_log_probs = torch.index_select(sum_log_probs, dim=0, index=active_beam_idx)

            if done: break
            decoder_input = enc_outputs[torch.arange(0, bpres_actions.shape[0], device=self.device), bpres_actions]

            bdyn_inputs = self.dyn_feat.make_dynamic_feat(inputs.expand(beam_idx.shape[0], -1, -1), bpresent_time, bpres_actions, dist_mat, active_beam_idx)
            emb2 = self.neuralnet.dyn_emb(bdyn_inputs)
            enc_inputs = torch.cat((emb1,emb2), dim=2)

            policy, dec_hidden, enc_outputs = self.neuralnet(enc_inputs, enc_outputs, adj_mask, decoder_input, dec_hidden, mask)

            future_actions, log_probs, beam_idx = self.select_actions(policy, sum_log_probs, mask, infer_type)

            # info update
            h_step = torch.index_select(dec_hidden[0], dim=0, index = beam_idx)
            c_step = torch.index_select(dec_hidden[1], dim=0, index = beam_idx)
            dec_hidden = (h_step,c_step)

            mask = torch.index_select(mask, dim=0, index=beam_idx)
            adj_mask = torch.index_select(adj_mask, dim=0, index=beam_idx)

            bpresent_time = torch.index_select(bpresent_time, dim=0, index=beam_idx)
            bpres_actions = torch.index_select(bpres_actions, dim=0, index=beam_idx)

            enc_outputs  = torch.index_select(enc_outputs, dim=0, index=beam_idx)

            emb1 = torch.index_select(emb1, dim=0, index=beam_idx)

            blactions = torch.index_select(blactions, dim=0, index=beam_idx)
            bllog_probs = torch.index_select(bllog_probs, dim=0, index=beam_idx)
            sum_log_probs = torch.index_select(sum_log_probs, dim=0, index=beam_idx)

            sum_log_probs += log_probs.squeeze(0).detach()

        return torch.cat(final_actions, dim=0), torch.cat(final_log_probs, dim=0)



    def select_actions(self, policy, sum_log_probs, mask, infer_type = 'stochastic'):

        beam_size, seq_size = policy.size()
        nzn  = torch.nonzero(mask, as_tuple=False).shape[0]
        sample_size = min(nzn,self.beam_size)

        ourlogzero = sys.float_info.min
        lpolicy = policy.masked_fill(mask == 0, ourlogzero).log()
        npolicy = sum_log_probs.unsqueeze(1) + lpolicy
        if infer_type == 'stochastic':
            nnpolicy = npolicy.exp().masked_fill(mask == 0, 0).view(1, -1)

            m = Categorical(nnpolicy)
            gact_ind = torch.multinomial(nnpolicy, sample_size)
            log_select =  m.log_prob(gact_ind)

        elif infer_type == 'greedy':
            nnpolicy = npolicy.exp().masked_fill(mask == 0, 0).view(1, -1)

            _ , gact_ind = nnpolicy.topk(sample_size, dim = 1)
            prob = policy.view(-1)[gact_ind]
            log_select =  prob.log()

        beam_id = torch.floor_divide(gact_ind, seq_size).squeeze(0)
        act_ind = torch.fmod(gact_ind, seq_size)

        return act_ind, log_select, beam_id

In [11]:
class RunEpisode(nn.Module):

    def __init__(self, neuralnet, args):
        super(RunEpisode, self).__init__()

        self.device = args.device
        self.neuralnet = neuralnet
        self.dyn_feat = DynamicFeatures(args)
        self.lookahead = Lookahead(args)
        self.mu = ModelUtils(args)

    def forward(self, binputs, bdata_scaled, start_time, dist_mat, infer_type):

        self.batch_size, sequence_size, input_size = binputs.size()

        h_0, c_0 = self.neuralnet.decoder.hidden_0

        dec_hidden = (h_0.expand(self.batch_size, -1), c_0.expand(self.batch_size, -1))

        mask = torch.ones(self.batch_size, sequence_size, device=self.device, requires_grad=False, dtype = torch.uint8)

        bpresent_time = start_time*torch.ones(self.batch_size, 1, device=self.device)

        llog_probs, lactions, lstep_mask, lentropy = [], [], [], []

        bpres_actions = torch.zeros(self.batch_size, dtype=torch.int64, device=self.device)

        batch_idx = torch.arange(0, self.batch_size, device=self.device)

        done, mask = self.mu.feasibility_control(binputs[batch_idx], mask, dist_mat, bpres_actions,
                                                 bpresent_time, batch_idx, first_step=True)

        adj_mask = self.lookahead.adjacency_matrix(binputs[batch_idx], mask, dist_mat, bpres_actions, bpresent_time)

        # encoder first forward pass
        bdyn_inputs = self.dyn_feat.make_dynamic_feat(binputs, bpresent_time, bpres_actions, dist_mat, batch_idx)
        emb1 = self.neuralnet.sta_emb(bdata_scaled)
        emb2 = self.neuralnet.dyn_emb(bdyn_inputs)
        enc_inputs = torch.cat((emb1,emb2), dim=2)

        _, _, enc_outputs = self.neuralnet(enc_inputs, enc_inputs, adj_mask, enc_inputs, dec_hidden, mask, first_step=True)

        decoder_input = enc_outputs[batch_idx, bpres_actions]

        done, mask = self.mu.feasibility_control(binputs[batch_idx], mask, dist_mat, bpres_actions, bpresent_time, batch_idx)
        adj_mask = self.lookahead.adjacency_matrix(binputs[batch_idx], mask, dist_mat, bpres_actions, bpresent_time)

        # encoder/decoder forward pass
        bdyn_inputs = self.dyn_feat.make_dynamic_feat(binputs, bpresent_time,
                                                      bpres_actions, dist_mat, batch_idx)
        emb2 = self.neuralnet.dyn_emb(bdyn_inputs)
        enc_inputs = torch.cat((emb1,emb2), dim=2)

        policy, dec_hidden, enc_outputs = self.neuralnet(enc_inputs, enc_outputs, adj_mask, decoder_input, dec_hidden, mask)

        lactions.append(bpres_actions)

        # Starting the trip
        while not done:

            future_actions, log_probs, entropy = self.select_actions(policy, infer_type)

            bpres_actions, bpresent_time, bstep_mask = self.mu.one_step_update(binputs, dist_mat, bpres_actions[batch_idx],
                                                                               future_actions, bpresent_time[batch_idx],
                                                                               batch_idx, self.batch_size)

            blog_probs = torch.zeros(self.batch_size, 1, dtype=torch.float32).to(self.device)
            blog_probs[batch_idx] = log_probs.unsqueeze(1)

            bentropy = torch.zeros(self.batch_size,1,dtype=torch.float32).to(self.device)
            bentropy[batch_idx] = entropy.unsqueeze(1)

            llog_probs.append(blog_probs)
            lactions.append(bpres_actions)
            lstep_mask.append(bstep_mask)
            lentropy.append(bentropy)

            done, mask = self.mu.feasibility_control(binputs[batch_idx], mask, dist_mat,
                                                     bpres_actions[batch_idx], bpresent_time[batch_idx],
                                                     batch_idx)

            if done: break
            sub_batch_idx = torch.nonzero(mask[batch_idx][:,-1], as_tuple=False).squeeze(1)

            batch_idx = torch.nonzero(mask[:,-1], as_tuple=False).squeeze(1)

            adj_mask = self.lookahead.adjacency_matrix(binputs[batch_idx], mask[batch_idx], dist_mat, bpres_actions[batch_idx], bpresent_time[batch_idx])

            #update decoder input and hidden
            decoder_input = enc_outputs[sub_batch_idx, bpres_actions[sub_batch_idx]]
            dec_hidden = (dec_hidden[0][sub_batch_idx], dec_hidden[1][sub_batch_idx])

            # encoder/decoder forward pass
            bdyn_inputs = self.dyn_feat.make_dynamic_feat(binputs, bpresent_time[batch_idx], bpres_actions[batch_idx], dist_mat, batch_idx)
            emb2 = self.neuralnet.dyn_emb(bdyn_inputs)
            enc_inputs = torch.cat((emb1[batch_idx],emb2), dim=2)

            policy, dec_hidden, enc_outputs = self.neuralnet(enc_inputs, enc_outputs[sub_batch_idx], adj_mask, decoder_input, dec_hidden, mask[batch_idx])

        return lactions, torch.cat(llog_probs, dim=1), torch.cat(lentropy, dim=1), torch.cat(lstep_mask, dim=1)


    def select_actions(self, policy, infer_type):

        if infer_type == 'stochastic':
            m = Categorical(policy)
            act_ind = m.sample()
            log_select =  m.log_prob(act_ind)
            poli_entro = m.entropy()
        elif infer_type == 'greedy':
            prob, act_ind = torch.max(policy, 1)
            log_select =  prob.log()
            poli_entro =  torch.zeros(self.batch_size, requires_grad=False).to(self.device)

        return act_ind, log_select, poli_entro


### Utils

In [12]:
from dotmap import DotMap

In [13]:
# config.py in the original code
cf = dict(
        BENCHMARK_INSTANCES_PATH = './data/benchmark/',
        GENERATED_INSTANCES_PATH = './data/generated/',
        RESULTS_PATH = './results'
)
cf = DotMap(cf)

In [14]:
# problem_config.py in the original code
pcf = DotMap(dict(
# Indices of instance data
X_COORDINATE_IDX = 0,
Y_COORDINATE_IDX = 1,
VIS_DURATION_TIME_IDX = 2,
OPENING_TIME_WINDOW_IDX = 3,
CLOSING_TIME_WINDOW_IDX = 4,
REWARD_IDX = 5,
ARRIVAL_TIME_IDX = 6,

# For generating instances
SAMP_DAY_FRAC_INF = 4/24,
UB_T_INIT_FRAC = 15/24,
LB_T_MAX_FRAC = 12/24,
CORR_SCORE_STD = 10,

MULTIPLE_SCORE = 1.1,

X_MAX = 100. # max square length (X_MAX)
))

In [15]:
def read_instance_data(instance_name, path):

    """reads instance data"""
    PATH_TO_BENCHMARK_INSTANCES = path

    benchmark_file = '{path_to_benchmark_instances}/{instance}.txt' \
                     .format(path_to_benchmark_instances=PATH_TO_BENCHMARK_INSTANCES,
                             instance=instance_name)

    dfile = open(benchmark_file)
    data = [[float(x) for x in line.split()] for line in dfile]
    dfile.close()
    return data


In [16]:
def eliminate_extra_cordeau_columns(instance_data):
    """Cordeau instances have extra columns in some rows. This function eliminates the extra columns.
    This will also correct position of total time in row 0 for all instances"""
    DATA_INIT_ROW = 2
    N_RELEVANT_FIRST_COLUMNS = 8
    N_RELEVANT_LAST_COLUMNS = 2

    return [s[:N_RELEVANT_FIRST_COLUMNS]+s[-N_RELEVANT_LAST_COLUMNS:] \
            for s in instance_data[DATA_INIT_ROW :]]


In [17]:
def parse_instance_data(instance_data):
    """parse instance data into dataframe"""

    OPENING_TIME_WINDOW_ABBREV_KEY = 'O'
    CLOSING_TIME_WINDOW_ABBREV_KEY = 'C'
    TOTAL_TIME_KEY = 'Total Time'
    COLUMN_NAMES_ABBREV = ['i', 'x', 'y', 'd', 'S', 'f', 'a', 'list', 'O', 'C']

    instance_data_clean = eliminate_extra_cordeau_columns(instance_data)
    df = pd.DataFrame(instance_data_clean, columns=COLUMN_NAMES_ABBREV)

    #add total time
    df[TOTAL_TIME_KEY] = 0
    df[TOTAL_TIME_KEY] = df.loc[0][CLOSING_TIME_WINDOW_ABBREV_KEY]

    return df


In [18]:
def test_n_vert_1(instance_data, instance_type):
    N_VERT_ROW = 0

    if instance_type=='Gavalas':
        N_VERT_COL = 3
        DATA_INIT_ROW = 1
    else:
        N_VERT_COL = 2
        DATA_INIT_ROW = 2

    n_vert = instance_data[N_VERT_ROW][N_VERT_COL]
    count_vert = len(instance_data)-(DATA_INIT_ROW+1)

    assert count_vert==n_vert, 'number of vertices doesnt match number of data rows'


def test_n_vert_2(instance_data, instance_type):
    N_VERT_ROW = 0
    if instance_type=='Gavalas':
        N_VERT_COL = 3
    else:
        N_VERT_COL = 2
    COLUMN_NAMES = ['vertex number', 'x coordinate', 'y coordinate',
                    'service duration or visiting time', 'profit of the location',
                    'not relevant 1', 'not relevant 2', 'not relevant 3',
                    'opening of time window', 'closing of time window']
    COLUMN_NAMES = [s.replace(' ', '_') for s in COLUMN_NAMES]

    VERTEX_NUMBER_COL = [i for i,n in enumerate(COLUMN_NAMES) if n=='vertex_number'][0]
    n_vert = instance_data[N_VERT_ROW][N_VERT_COL]
    last_vert_number = instance_data[-1][VERTEX_NUMBER_COL]

    assert last_vert_number==n_vert, 'number of vertices doesnt match vertice count of last row'


def test_n_vert_3(instance_data, instance_type):
    if instance_type=='Gavalas':
        N_DAYS_INDEX = 1
        n_days = int(np.array(instance_data[0])[N_DAYS_INDEX])
        assert n_days==1, 'not a single tour/1 day instance'
    else:
        pass

In [19]:
def get_distance_matrix(instance_df, instance_type):
    """
    Distances between locations were rounded down to the first decimal
    for the Solomon instances and to the second decimal for the instances of Cordeau and Gavalas.
    """

    if instance_type in ['Solomon']:
        n_digits = 10.0

    elif instance_type in ['Cordeau', 'Gavalas']:
        n_digits = 100.0

    n = instance_df.shape[0]
    distm = np.zeros((n,n))
    x = instance_df.x.values
    y = instance_df.y.values

    for i in range(0, n-1):
        for j in range(i+1, n):
            distm[i,j] = np.floor(n_digits*(np.sqrt((x[i]-x[j])**2+(y[i]-y[j])**2)))/n_digits
            distm[j,i] = distm[i,j]

    return distm


In [20]:
def get_instance_df(instance_name, path, instance_type):

    """combine read instance, tests and parse to dataframe"""

    OPENING_TIME_WINDOW_ABBREV_KEY = 'O'
    CLOSING_TIME_WINDOW_ABBREV_KEY = 'C'
    TOTAL_TIME_KEY = 'Total Time'


    COLUMN_NAMES = ['vertex number', 'x coordinate', 'y coordinate',
    'service duration or visiting time', 'profit of the location',
    'not relevant 1', 'not relevant 2', 'not relevant 3',
    'opening of time window', 'closing of time window']
    COLUMN_NAMES = [s.replace(' ', '_') for s in COLUMN_NAMES]
    COLUMN_NAMES_ABBREV = ['i', 'x', 'y', 'd', 'S', 'f', 'a', 'list', 'O', 'C']
    VERTEX_NUMBER_COL = [i for i,n in enumerate(COLUMN_NAMES) if n=='vertex_number'][0]
    COLS_OF_INT = ['i', 'x', 'y', 'd', OPENING_TIME_WINDOW_ABBREV_KEY,
                   CLOSING_TIME_WINDOW_ABBREV_KEY, 'S', TOTAL_TIME_KEY]
    COLS_OF_INT_NEW_NAMES = ['i', 'x', 'y', 'duration', 'ti', 'tf', 'prof', TOTAL_TIME_KEY]

    standard2newnames_dict =  dict(((c, ca) for c, ca in zip(COLS_OF_INT, COLS_OF_INT_NEW_NAMES)))

    instance_data = read_instance_data(instance_name, path)

    # run tests
    test_n_vert_1(instance_data, instance_type)
    test_n_vert_2(instance_data, instance_type)
    # test if it's a single day (we are not considering TOPTW instances)
    test_n_vert_3(instance_data, instance_type)

    if instance_type=='Gavalas':
        df = parse_instance_data_Gavalas(instance_data)
    else:
        df = parse_instance_data(instance_data)

    #change column names
    COLS_OF_INT_NEW_NAMES = [standard2newnames_dict[s] for s in COLS_OF_INT]
    df_ = df[COLS_OF_INT].copy()
    df_.columns = COLS_OF_INT_NEW_NAMES
    df_['inst_name'] = instance_name
    df_['real_or_val'] = 'real'

    df_ = df_.append(df_.loc[0])
    return df_

In [21]:
def get_real_data(args, phase='train'):

    df = get_instance_df(args.instance, cf.BENCHMARK_INSTANCES_PATH,
                         args.instance_type)
    dist_mat = get_distance_matrix(df, args.instance_type)
    inp_real = df[['x', 'y', 'duration', 'ti', 'tf', 'prof', 'Total Time']].values

    if phase=='train':
        inp_real = [(torch.FloatTensor(inp_real).to(args.device),
                torch.tensor(inp_real[0, pcf.OPENING_TIME_WINDOW_IDX]).to(args.device),
                torch.FloatTensor(dist_mat).to(args.device))]

        new_inp_real = [(args.instance, inp_real[0])]
        return new_inp_real
    else:
        inp_real = [(torch.FloatTensor(inp_real).to(args.device),
                 torch.FloatTensor(dist_mat).to(args.device))]
        return inp_real

### Sampling Norm Utils

In [22]:
def instance_dependent_norm_const(instance_raw_data):
    day_duration = int(instance_raw_data[:, pcf.CLOSING_TIME_WINDOW_IDX].max().item())
    t_max_real = int(instance_raw_data[0, pcf.ARRIVAL_TIME_IDX].item()) # max instance arrival time
    arrival_time_val_ub = t_max_real+int(pcf.SAMP_DAY_FRAC_INF*day_duration)
    Tmax = int(max(day_duration, arrival_time_val_ub)) # max possible time value
    Smax = int(torch.round(pcf.MULTIPLE_SCORE*instance_raw_data[1:-1, pcf.REWARD_IDX].max()).item()) # max score

    return Tmax, Smax


In [23]:
def data_scaler(data, norm_dic):
    datan = data.clone()
    datan[:, pcf.X_COORDINATE_IDX] /= pcf.X_MAX
    datan[:, pcf.Y_COORDINATE_IDX] /= pcf.X_MAX
    datan[:, pcf.VIS_DURATION_TIME_IDX] /= (datan[:, pcf.VIS_DURATION_TIME_IDX].max())
    datan[:, pcf.OPENING_TIME_WINDOW_IDX] /= norm_dic['Tmax']
    datan[:, pcf.CLOSING_TIME_WINDOW_IDX ] /= norm_dic['Tmax']
    datan[:, pcf.REWARD_IDX] /= norm_dic['Smax']
    datan[:, pcf.ARRIVAL_TIME_IDX] /= norm_dic['Tmax']

    return datan

### Train Utils

In [24]:
def reward_fn(data, sample_solution, device):
    """
    Returns:
        Tensor of shape [batch_size] containing rewards
    """

    batch_size = sample_solution[0].shape[0]
    tour_reward = torch.zeros(batch_size, device=device)

    for act_id in sample_solution:
        tour_reward += data[act_id, pcf.REWARD_IDX].squeeze(0)

    return tour_reward


### Inference Utils

In [25]:
def run_single(inst_data, norm_dic, start_time, dist_mat, args, model,
               which_inf=None):


    saved_model_path = args.load_w_dir +'/model_' + str(args.saved_model_epoch) + '.pkl'
    model._load_model_weights(saved_model_path, args.device)


    tic = time.time()
    if which_inf=='bs':
        run_episode_inf = BeamSearch(model, args).eval()
        route, score = bs_inference(inst_data, norm_dic, start_time, dist_mat,
                                    args, run_episode_inf)

    elif which_inf=='gr':
        run_episode_inf = RunEpisode(model, args).eval()
        route, score = gr_inference(inst_data, norm_dic, start_time, dist_mat,
                                    args, run_episode_inf)

    elif which_inf=='as_bs':

        saved_model_path = args.load_w_dir +'/model_' + str(args.saved_model_epoch) + '.pkl'
        model._load_model_weights(saved_model_path, args.device)
        run_episode_train = RunEpisode(model, args)

        run_episode_inf = BeamSearch(model, args).eval()
        inp_data = (inst_data, start_time, dist_mat)
        route, score = as_bs_inference(inp_data, norm_dic, args,
                                       run_episode_train, run_episode_inf)
    toc = time.time()

    output = dict([('score', score), ('route', route), ('inf_time', toc-tic)])

    return output

In [26]:
def bs_inference(inst_data, norm_dic, start_time, dist_mat, args, run_episode):

    data_scaled = data_scaler(inst_data, norm_dic)
    binst_data, bdata_scaled = inst_data.unsqueeze(0), data_scaled.unsqueeze(0)

    nb = args.max_beam_number
    with torch.no_grad():
        seq, _ = run_episode(binst_data, bdata_scaled, start_time, dist_mat, 'greedy', nb)

    seq_list = [ seq[:,k] for k in range(seq.shape[1])]
    rewards = reward_fn(inst_data, seq_list, args.device)
    maxrew, idx_max = torch.max(rewards, 0)
    score = maxrew.item()

    route =  [0] + [val.item() for val in seq[idx_max] if val.item() != 0]
    route[-1] = 0
    return route, score


### Features Utils

In [27]:
class DynamicFeatures():

    def __init__(self, args):
        super(DynamicFeatures, self).__init__()

        self.arrival_time_idx = pcf.ARRIVAL_TIME_IDX
        self.opening_time_window_idx = pcf.OPENING_TIME_WINDOW_IDX
        self.closing_time_window_idx = pcf.CLOSING_TIME_WINDOW_IDX
        self.device = args.device

    def make_dynamic_feat(self, data, current_time, current_poi_idx, dist_mat, batch_idx):

        num_dyn_feat = 8
        _ , sequence_size, input_size  = data.size()
        batch_size = batch_idx.shape[0]

        dyn_feat = torch.ones(batch_size, sequence_size, num_dyn_feat).to(self.device)

        tour_start_time = data[0, 0, self.opening_time_window_idx]
        max_tour_duration = data[0, 0, self.arrival_time_idx] - tour_start_time
        arrive_j_times = current_time + dist_mat[current_poi_idx]

        dyn_feat[:, :, 0] = (data[batch_idx, :, self.opening_time_window_idx] - current_time) / max_tour_duration
        dyn_feat[:, :, 1] = (data[batch_idx, :, self.closing_time_window_idx] - current_time) / max_tour_duration
        dyn_feat[:, :, 2] = (data[batch_idx, :, self.arrival_time_idx] - current_time) / max_tour_duration
        dyn_feat[:, :, 3] = (current_time - tour_start_time) / max_tour_duration


        dyn_feat[:, :, 4] = (arrive_j_times - tour_start_time) / max_tour_duration
        dyn_feat[:, :, 5] = (data[batch_idx, :, self.opening_time_window_idx] - arrive_j_times) / max_tour_duration
        dyn_feat[:, :, 6] = (data[batch_idx, :, self.closing_time_window_idx] - arrive_j_times) / max_tour_duration
        dyn_feat[:, :, 7] = (data[batch_idx, :, self.arrival_time_idx] - arrive_j_times) / max_tour_duration

        return dyn_feat


### Solution Construction

In [28]:
class Lookahead():
    def __init__(self, args):
        super(Lookahead, self).__init__()

        self.device = args.device
        self.opening_time_window_idx = pcf.OPENING_TIME_WINDOW_IDX
        self.closing_time_window_idx = pcf.CLOSING_TIME_WINDOW_IDX
        self.vis_duration_time_idx = pcf.VIS_DURATION_TIME_IDX
        self.arrival_time_idx = pcf.ARRIVAL_TIME_IDX

    def adjacency_matrix(self, braw_inputs, mask, dist_mat, pres_act, present_time):
        # feasible neighborhood for each node
        maskk = mask.clone()
        step_batch_size, npoints = mask.shape

        #one step forward update
        arrivej = dist_mat[pres_act] + present_time
        farrivej = arrivej.view(step_batch_size, npoints)
        tw_start = braw_inputs[:, :, self.opening_time_window_idx]
        waitj = torch.max(torch.FloatTensor([0.0]).to(self.device), tw_start-farrivej)
        durat = braw_inputs[:, : , self.vis_duration_time_idx]

        fpresent_time = farrivej + waitj + durat
        fpres_act = torch.arange(0, npoints, device=self.device).expand(step_batch_size, -1)

        # feasible neighborhood for each node
        adj_mask = maskk.unsqueeze(1).repeat(1, npoints, 1)
        arrivej = dist_mat.expand(step_batch_size, -1, -1) + fpresent_time.unsqueeze(2)
        waitj = torch.max(torch.FloatTensor([0.0]).to(self.device), tw_start.unsqueeze(2)-arrivej)

        tw_end = braw_inputs[:, :, self.closing_time_window_idx]
        ttime = braw_inputs[:, 0, self.arrival_time_idx]

        dlast = dist_mat[:, -1].unsqueeze(0).expand(step_batch_size, -1)

        c1 = arrivej + waitj <= tw_end.unsqueeze(1)
        c2 = arrivej + waitj + durat.unsqueeze(1) + dlast.unsqueeze(1) <= ttime.unsqueeze(1).unsqueeze(1).expand(-1, npoints, npoints)
        adj_mask = adj_mask * c1 * c2

        # self-loop
        idx = torch.arange(0, npoints, device=self.device).expand(step_batch_size, -1)
        adj_mask[:, idx, idx] = 1

        return adj_mask

In [29]:
class ModelUtils():
    def __init__(self, args):
        super(ModelUtils, self).__init__()

        self.device = args.device
        self.opening_time_window_idx = pcf.OPENING_TIME_WINDOW_IDX
        self.closing_time_window_idx = pcf.CLOSING_TIME_WINDOW_IDX
        self.vis_duration_time_idx = pcf.VIS_DURATION_TIME_IDX
        self.arrival_time_idx = pcf.ARRIVAL_TIME_IDX

    def feasibility_control(self, braw_inputs, mask, dist_mat, pres_act, present_time, batch_idx, first_step=False):

        done = False
        maskk = mask.clone()
        step_batch_size = batch_idx.shape[0]

        arrivej = dist_mat[pres_act] + present_time
        waitj = torch.max(torch.FloatTensor([0.0]).to(self.device), braw_inputs[:, :, self.opening_time_window_idx]-arrivej)

        c1 = arrivej + waitj <= braw_inputs[:, :, self.closing_time_window_idx]
        c2 = arrivej + waitj + braw_inputs[:, :, self.vis_duration_time_idx] + dist_mat[:, -1] <= braw_inputs[0, 0, self.arrival_time_idx]

        if not first_step:
            maskk[batch_idx, pres_act] = 0

        maskk[batch_idx] = maskk[batch_idx] * c1 * c2

        if maskk[:, -1].any() == 0:
            done = True
        return done, maskk

    
    def one_step_update(self, raw_inputs_b, dist_mat, pres_action, future_action, present_time, batch_idx, batch_size):

        present_time_b = torch.zeros(batch_size, 1, device=self.device)
        pres_actions_b = torch.zeros(batch_size, dtype=torch.int64, device=self.device)
        step_mask_b = torch.zeros(batch_size, 1, device=self.device, requires_grad=False, dtype=torch.bool)

        arrive_j = dist_mat[pres_action, future_action].unsqueeze(1) + present_time
        wait_j = torch.max(torch.FloatTensor([0.0]).to(self.device),
                           raw_inputs_b[batch_idx, future_action, self.opening_time_window_idx].unsqueeze(1)-arrive_j)
        present_time = arrive_j + wait_j + raw_inputs_b[batch_idx, future_action, self.vis_duration_time_idx].unsqueeze(1)

        present_time_b[batch_idx] = present_time

        pres_actions_b[batch_idx] = future_action
        step_mask_b[batch_idx] = 1

        return pres_actions_b, present_time_b, step_mask_b

### Run the model here

In [30]:
args = dict(
    batch_size=32, 
    beta=0.01, 
    device='cpu', 
    debug=False, 
    device_name='cpu', 
    ff_dim=256, 
    generated=False, 
    infe_type='bs', 
    instance='pr01', 
    instance_type='Cordeau', 
    load_w_dir='./results/pr01/model_w/model_article_uni_samp', 
    lr=1e-05, 
    map_location={'cpu': 'cpu'}, 
    max_beam_number=128, 
    max_grad_norm=1, 
    model_name='article', 
    n_heads=8, 
    n_layers=2, 
    ndfeatures=8, 
    nepocs=128, 
    nfeatures=7, 
    nprint=1, 
    rnn_hidden=128, 
    sample_type='uni_samp', 
    saved_model_epoch=500000, 
    seed=2925, 
    use_checkpoint=False, 
    val_dir='./data/generated//pr01', val_set_pt_file='inp_val_uni_samp.pt') 

In [31]:
args = DotMap(args)

In [32]:
inp_real = get_real_data(args, phase='inference')
raw_data, raw_distm = inp_real[0]

In [33]:
start_time = raw_data[0, pcf.OPENING_TIME_WINDOW_IDX]

In [34]:
# get Tmax and Smax
norm_dic = {}
Tmax, Smax = instance_dependent_norm_const(raw_data)
norm_dic = {'Tmax': Tmax, 'Smax': Smax}

In [35]:
performance_scores = []

# args.nfeatures : Number of Static features
# args.ndfeatures: Number of Dynamic features
# args.rnn_hidden: Number of Hidden features 

pointer_net = RecPointerNetwork(args.nfeatures, args.ndfeatures,
                          args.rnn_hidden, args).to(args.device).eval()


In [36]:
# ---------------------------------
# inference
# ---------------------------------

if not args.generated:
    print('Infering route for benchmark instance...')
    output =  run_single(raw_data, norm_dic, start_time, raw_distm, args,
                            pointer_net, which_inf=args.infe_type)

else:
    inp_val = u.get_val_data(args, phase='inference')
    print('Infering routes for {num_inst} generated instances...' \
                .format(num_inst=len(inp_val)))
    outputs =  iu.run_multiple(inp_val, norm_dic, args, pointer_net,
                               which_inf=args.infe_type)

Infering route for benchmark instance...


In [37]:
# ---------------------------------
# Log results
# ---------------------------------
N_DASHES = 40
if args.infe_type in ['gr', 'bs', 'as_bs']:

    print(N_DASHES*'-')
    if not args.generated:
        print('route: {route}'.format(route=output['route']))
        print('total score: {total_score}'\
                    .format(total_score=int(output['score'])))
        inference_time_ms = int(1000*output['inf_time'])
        print('inference time: {inference_time} ms'\
                    .format(inference_time=inference_time_ms))

    else:
        df_out = pd.DataFrame(outputs)
        average_total_score = round(df_out.score.mean(), 2)
        average_inf_time_ms = int(1000*df_out.inf_time.mean())
        print('average total score: {average_total_score}' \
                    .format(average_total_score=average_total_score))
        print('average inference time: {average_inference_time} ms' \
                    .format(average_inference_time=average_inf_time_ms))
    print(N_DASHES*'-')


----------------------------------------
route: [0, 9, 24, 47, 12, 38, 30, 2, 32, 37, 10, 45, 11, 28, 1, 16, 36, 31, 35, 34, 22, 7, 0]
total score: 308
inference time: 9518 ms
----------------------------------------
